# Overview
This will use the *new* tropical data API to export survey data. To begin we just export all data for each project

In [2]:
import requests
import csv

USER="<PUT USERNAME HERE>"
PASS="<PUT PASSWORD HERE>"

In [31]:
# from here: https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
        
    block = int(round(bar_length * progress))
    
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

## Step 1: Get available projects

In [32]:
# Get projects
r1 = requests.get('https://www.tropicaldata.org/api/projects', auth=(USER, PASS))
projects = r1.json()

In [33]:
project_ids = []
for project in projects:
    #print(project['id'])
    project_ids.append(str(project['id']))
    
project_ids = ','.join(project_ids)
print(project_ids)

109,48,79,90,72,82,106


In [43]:
# Get the total numvber of records, this is needed for the paging
r2 = requests.get('https://www.tropicaldata.org/api/analysis?project_ids='+project_ids+'&date_completed=2020-01-01T00:00:00', auth=(USER, PASS))
analysis_data = r2.json()
total_records = analysis_data['total']
print("there are {}".format(total_records)+" records")

there are 491 records


In [39]:
import math

results = []
for n in range(math.ceil(total_records/100) ): # need to call this enough times to get all of the records. if 2.3 call it 3 times
    offset= n*100 # 100 is the number of records in the response
    r2 = requests.get('https://www.tropicaldata.org/api/analysis?project_ids='+project_ids+'&date_completed=2015-01-01T00:00:00&offset='+str(offset), auth=(USER, PASS))
    analysis_data = r2.json()
    results = results + analysis_data['data']
    print('getting {}'.format(len(analysis_data['data'])) + " records of {}".format(total_records) )

getting 100 records of 491
getting 100 records of 491
getting 100 records of 491
getting 100 records of 491
getting 91 records of 491


## Step 2: Load in current orgs from database.
This will be used later to determine there is a match in the NTDDB

In [1]:
import numpy as np
import pandas as pd

ntd_orgs = pd.read_csv('../ntddb_orgs.csv')

In [2]:
ntd_orgs.head(10)

,woreda,woreda uid,zone,zone id,region,region id
0,Abadir,ORG00000302,Harare,ORG00000301,Harare,ORG00000299
1,Abala,ORG00000029,Awsa,ORG00000014,Afar,ORG00000013
2,Abaya,unZLOLtOisg,Borena,ORG00000361,Oromia,ORG00000311
3,Abaya,ORG00000635,West Guji,ORG00000631,Oromia,ORG00000311
4,Abay Comen,ORG00000484,Horo Guduru,ORG00000482,Oromia,ORG00000311
5,Abe Dongoro,ORG00000483,Horo Guduru,ORG00000482,Oromia,ORG00000311
6,Abergele,ORG00000194,Wag Hemra,ORG00000193,Amhara,ORG00000053
7,Abeshge,ORG00000723,Gurage,ORG00000719,SNNPR,ORG00000668
8,Abi Adi,ORG00000942,Central Tigray,ORG00000935,Tigray,ORG00000934
9,Abichugna,ORG00000549,North Shoa Zone,ORG00000543,Oromia,ORG00000311


In [3]:
# this file contains the alternate names for woredas
updated_names = pd.read_excel('mapping.xlsx')

#ntd_orgs = ntd_orgs.set_index('Woreda ID').join(updated_names.set_index('Woreda ID'))
ntd_orgs = ntd_orgs.join(updated_names.set_index('woreda uid'), on='woreda uid')

In [47]:
ntd_orgs.head(10)

,woreda,woreda uid,zone,zone id,region,region id,alternate name
0,Abadir,ORG00000302,Harare,ORG00000301,Harare,ORG00000299,NaN
1,Abala,ORG00000029,Awsa,ORG00000014,Afar,ORG00000013,NaN
2,Abaya,unZLOLtOisg,Borena,ORG00000361,Oromia,ORG00000311,NaN
3,Abaya,ORG00000635,West Guji,ORG00000631,Oromia,ORG00000311,NaN
4,Abay Comen,ORG00000484,Horo Guduru,ORG00000482,Oromia,ORG00000311,NaN
5,Abe Dongoro,ORG00000483,Horo Guduru,ORG00000482,Oromia,ORG00000311,NaN
6,Abergele,ORG00000194,Wag Hemra,ORG00000193,Amhara,ORG00000053,NaN
7,Abeshge,ORG00000723,Gurage,ORG00000719,SNNPR,ORG00000668,NaN
8,Abi Adi,ORG00000942,Central Tigray,ORG00000935,Tigray,ORG00000934,NaN
9,Abichugna,ORG00000549,North Shoa Zone,ORG00000543,Oromia,ORG00000311,NaN


In [48]:
def find_admin3(tropical_data_district):

    for index, ntd_row in ntd_orgs.iterrows():
        
        if ntd_row['woreda'].lower() == tropical_data_district.lower():
            return ntd_row['woreda uid']
        
        elif pd.notna(ntd_row['alternate name']) \
        and ntd_row['alternate name'].lower() == tropical_data_district.lower():
            return ntd_row['woreda uid']
        

In [49]:
# test this out
match = find_admin3("Ayehu9")
if match:
    print("found. and is {}".format( match) )
else:
    print("not found")

not found


In [50]:
find_admin3("Ayehu")

'ORG00000055'

### This step is optional

In [51]:
# Write out to a CSV file
with open('tropicaldata-ous.csv', 'w',  newline='') as analysis_data_export:
    
    fieldnames = ['project_id', 'admin1_name','admin2_name', 'admin3_name','date_completed', 'reasonable match in ntddb']
    writer = csv.DictWriter(analysis_data_export, fieldnames=fieldnames)
    writer.writeheader()
    
    for index,record in enumerate(results):
        
        matched = 'No'
        if find_admin3(record['admin3_name']):
            matched = 'Yes'
             
        writer.writerow({
            'project_id': record['project_id'],
            'admin1_name': record['admin1_name'],
            'admin2_name': record['admin2_name'],
            'admin3_name' : record['admin3_name'],
            'date_completed' : record['date_completed'],
            'reasonable match in ntddb' : matched
        })
        
        update_progress(index / len(results))
    

Progress: [####################] 99.8%


In [52]:
index

490

## Step 3: Write the data out to a file

In [53]:
import numpy as np
import pandas as pd

# write out to these to files
tropicaldata_df = pd.DataFrame(results)
tropicaldata_df.to_csv('tropicaldata-export.csv', index=True)

## Step 4: Write out the import file 
see Jeremiahas email from May 12

In [54]:
results[0]

{'admin0_name': 'Ethiopia',
 'admin1_name': 'Amhara',
 'admin2_name': 'Awi',
 'admin3_name': 'Ankesha Guagusa',
 'adult_abs': 470,
 'adult_enum': 2409,
 'adult_ex': 1913,
 'adult_female_abs': 119,
 'adult_female_enum': 1292,
 'adult_female_ex': 1157,
 'adult_female_oth': 0,
 'adult_female_refu': 16,
 'adult_female_trich_bilat': 12,
 'adult_female_trich_ct': 47,
 'adult_female_trich_loweronly': None,
 'adult_female_trich_postop_bilat': 2,
 'adult_female_trich_postop_eyes': 9,
 'adult_female_trich_unmanaged_ct': 28,
 'adult_female_trichts_bilat': 10,
 'adult_female_trichts_ct': 39,
 'adult_female_trichts_postop_bilat': 1,
 'adult_female_trichts_postop_eyes': 6,
 'adult_female_trichts_unmanaged_ct': 23,
 'adult_female_tt_bilat': None,
 'adult_female_tt_ct': None,
 'adult_female_tt_postop_bilat': None,
 'adult_female_tt_postop_eyes': None,
 'adult_female_tt_unmanaged_bilat': None,
 'adult_female_tt_unmanaged_ct': None,
 'adult_oth': 0,
 'adult_refu': 26,
 'adult_trich_bilat': 15,
 'adult_t

In [55]:
import dateutil.parser

datavalues = []
unmaped_ous = set()

for index, record in enumerate(results):
    
    # only import data for woredas that can be found
    match = find_admin3(record['admin3_name'])
    
    if match:
        
        #event_date = dateutil.parser.parse(record["analysis_date"])
        event_date = dateutil.parser.parse(record["date_completed"])
        
         # TF
        if "tf_adj" in record and record["tf_adj"] is not None:
           
            element = {
                "dataElement": "akcEiBEVnD1",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["tf_adj"] * 100,
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
                }
            
            datavalues.append(element)
        
        
        # TT
        if "tt_adj" in record and record["tt_adj"] is not None:   
             
            element = {
                "dataElement": "PR3XBXGC3dL",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["tt_adj"] * 100,
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
        # child_enum - Number of 1-9 year olds enumerated (regardless of examination status)
        if "child_enum" in record and record["child_enum"] is not None:   
             
            element = {
                "dataElement": "V2MPHZPFPj0",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["child_enum"],
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
        
        # child_ex - Number of 1-9 year olds examined
        if "child_ex" in record and record["child_ex"] is not None:   
             
            element = {
                "dataElement": "jLBmISBEYEl",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["child_ex"],
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
        # child_ex - Number of 15+ year olds enumerated
        if "adult_enum" in record and record["adult_enum"] is not None:   
             
            element = {
                "dataElement": "si6a8M15CM2",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["adult_enum"],
                "categoryOptionCombo": "HllvX50cXC0",
                 "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
         # child_ex - Number of 15+ year olds examined
        if "adult_ex" in record and record["adult_ex"] is not None:   
             
            element = {
                "dataElement": "v7CgLWmGb7y",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["adult_ex"],
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
         # child_ex - Number of 15+ year olds examined
        if "survey_type" in record and record["survey_type"] is not None:   
             
            element = {
                "dataElement": "e2hJK8SKZ1v",
                "period": event_date.year, 
                "orgUnit": match,
                "value": record["survey_type"],
                "categoryOptionCombo": "HllvX50cXC0",
                "attributeOptionCombo":"T2Ll5J1zITu"
            }
            datavalues.append(element)
            
              
    else:
        unmaped_ous.add(record['admin3_name'])
        
    update_progress( (index+1) / len(results))
        

Progress: [####################] 100.0%


In [56]:
# print what we know
print("Of the potential {}".format(len(results)) +" records from the API to be loaded in, there are {}"\
      .format(len(datavalues)) + " yearly forecasts to be imported and {}"\
      .format(len(unmaped_ous)) + " woredas that could not be mapped")


Of the potential 491 records from the API to be loaded in, there are 2924 yearly forecasts to be imported and 16 woredas that could not be mapped


In [57]:
import json

# Add these things
data = {}
data['dataValues'] = datavalues

with open('tropicaldata-for-importing.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=4)

### Import the file
In the Import Export app, select Data Import and this select this file generated above